In [ ]:
# random proxy taken from https://codelike.pro/create-a-crawler-with-rotating-ip-proxy-in-python/

In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import random

In [4]:
urls = ['https://www.basketball-reference.com/boxscores/?month=01&day=18&year=2019',
        'https://www.basketball-reference.com/boxscores/?month=01&day=19&year=2019']

In [5]:
k = 3

In [14]:
site_url = 'https://www.basketball-reference.com'

In [6]:
url_html = urlopen(urls[0])


In [47]:
soup = BeautifulSoup(url_html)



In [15]:
def fluff_number(x, digits = 2):
    y = str(x)
    gap = digits - len(str(x))
    if gap > 0:
        for i in range(gap):
            y = '0' + y
    return(y)

In [36]:
def get_proxies(K = 100):
    ua = UserAgent() # From here we generate a random user agent
    proxies = [] # Will contain proxies [ip, port]

    # Retrieve latest proxies
    proxies_req = Request('https://www.sslproxies.org/')
    proxies_req.add_header('User-Agent', ua.random)
    proxies_doc = urlopen(proxies_req).read().decode('utf8')

    soup = BeautifulSoup(proxies_doc, 'html.parser')
    proxies_table = soup.find(id='proxylisttable')

  # Save proxies in the array
    for row in proxies_table.tbody.find_all('tr'):
        proxies.append({
            'ip':   row.find_all('td')[0].string,
            'port': row.find_all('td')[1].string 
      })
    proxy_index = random_proxy()
    proxy = proxies[proxy_index]
    for n in range(1, K):
        req = Request('http://icanhazip.com')
        req.set_proxy(proxy['ip'] + ':' + proxy['port'], 'http')

        # Every k requests, generate a new proxy
        if n % 3 == 0:
            proxy_index = random_proxy()
            proxy = proxies[proxy_index]

        # Make the call
        try:
            my_ip = urlopen(req).read().decode('utf8')
            print('#' + str(n) + ': ' + my_ip)
        except: # If error, delete this proxy and find another one
            del proxies[proxy_index]
            print('Proxy ' + proxy['ip'] + ':' + proxy['port'] + ' deleted.')
            proxy_index = random_proxy()
            proxy = proxies[proxy_index]
    return(proxies)


In [41]:
proxies = get_proxies()

#1: 103.6.104.104

#2: 103.6.104.104

#3: 103.6.104.104

Proxy 95.38.209.33:50725 deleted.
Proxy 213.181.199.252:40745 deleted.
Proxy 1.20.100.195:30937 deleted.
Proxy 101.255.56.14:31926 deleted.
Proxy 5.45.64.97:3128 deleted.
Proxy 200.7.205.194:57712 deleted.
Proxy 94.73.217.125:40858 deleted.
#11: 167.99.59.202

#12: 167.99.59.202

Proxy 85.175.226.106:57396 deleted.
Proxy 190.152.0.130:42956 deleted.
Proxy 77.120.163.87:58203 deleted.
Proxy 113.23.179.114:58465 deleted.
#17: 157.230.83.15

#18: 157.230.83.15

Proxy 1.186.192.242:41834 deleted.
Proxy 103.29.184.44:80 deleted.
Proxy 111.92.241.194:33013 deleted.
#22: 1.20.99.174

#23: 1.20.99.174

#24: 1.20.99.174

#25: 128.199.86.146

#26: 128.199.86.146



IndexError: list index out of range

In [62]:
def get_games_from_date(month, day, year):
    site_url = 'http://www.basketball-reference.com'
    u = 'http://www.basketball-reference.com/boxscores/?'
    u += 'month=' + fluff_number(month)
    u += '&day=' + fluff_number(day)
    u += '&year=' + str(year)
    print(u)
    proxy_index = random_proxy()
    proxy = proxies[proxy_index]
    print(proxy)
    req = Request(u)
    req.set_proxy(proxy['ip'] + ':' + proxy['port'], 'http')
    html = urlopen(req)
    soup = BeautifulSoup(html)
    links = [site_url + x['href'] for x in soup.find_all('a', text = 'Box Score')]
    return(links)

In [114]:
def get_stat_from_tr(tr):
    return({tr.find('th')['csk']: [{x['data-stat']: x.text} for x in tr.find_all('td')]})

In [135]:
def get_table_from_game(u):
    proxy_index = random_proxy()
    proxy = proxies[proxy_index]

    req = Request(u)
    req.set_proxy(proxy['ip'] + ':' + proxy['port'], 'http')

    html = urlopen(req)
    soup = BeautifulSoup(html)
    # Get all of the rows whose first header contains the csk attribute
    trs = [tr for tr in soup.find_all('tr') if tr.find('th').has_attr('csk')]
    
    data = [get_stat_from_tr(tr) for tr in trs]
    return(data)

In [136]:
data = get_table_from_game('http://www.basketball-reference.com/boxscores/201901190ATL.html')

In [63]:
get_games_from_date(1, 19, 2019)

http://www.basketball-reference.com/boxscores/?month=01&day=19&year=2019
{'ip': '95.158.62.112', 'port': '44226'}


['http://www.basketball-reference.com/boxscores/201901190ATL.html',
 'http://www.basketball-reference.com/boxscores/201901190CHI.html',
 'http://www.basketball-reference.com/boxscores/201901190CHO.html',
 'http://www.basketball-reference.com/boxscores/201901190DEN.html',
 'http://www.basketball-reference.com/boxscores/201901190DET.html',
 'http://www.basketball-reference.com/boxscores/201901190HOU.html',
 'http://www.basketball-reference.com/boxscores/201901190IND.html',
 'http://www.basketball-reference.com/boxscores/201901190ORL.html',
 'http://www.basketball-reference.com/boxscores/201901190PHI.html',
 'http://www.basketball-reference.com/boxscores/201901190TOR.html']

In [2]:
# Main function
def scrape(u):
    
    ua = UserAgent() # From here we generate a random user agent
    proxies = [] # Will contain proxies [ip, port]

    # Retrieve latest proxies
    proxies_req = Request('https://www.sslproxies.org/')
    proxies_req.add_header('User-Agent', ua.random)
    proxies_doc = urlopen(proxies_req).read().decode('utf8')

    soup = BeautifulSoup(proxies_doc, 'html.parser')
    proxies_table = soup.find(id='proxylisttable')
a
  # Save proxies in the array
    for row in proxies_table.tbody.find_all('tr'):
        proxies.append({
            'ip':   row.find_all('td')[0].string,
            'port': row.find_all('td')[1].string 
      })

  # Choose a random proxy
    proxy_index = random_proxy()
    proxy = proxies[proxy_index]

    for n in range(1, 100):
        req = Request(u)
        req.set_proxy(proxy['ip'] + ':' + proxy['port'], 'http')

        # Every k requests, generate a new proxy
        if n % k == 0:
            proxy_index = random_proxy()
            proxy = proxies[proxy_index]

        # Make the call
        try:
            my_ip = urlopen(req).read().decode('utf8')
            print('#' + str(n) + ': ' + my_ip)
        except: # If error, delete this proxy and find another one
            del proxies[proxy_index]
            print('Proxy ' + proxy['ip'] + ':' + proxy['port'] + ' deleted.')
            proxy_index = random_proxy()
            proxy = proxies[proxy_index]

# Retrieve a random index proxy (we need the index to delete it if not working)
def random_proxy():
    return random.randint(0, len(proxies) - 1)

In [3]:
if __name__ == '__main__':
  main()

Proxy 41.33.31.38:40581 deleted.
#2: 157.230.83.203

#3: 157.230.83.203

#4: 157.230.83.203

#5: 157.230.83.203

#6: 157.230.83.203

#7: 157.230.83.203

#8: 157.230.83.203

#9: 157.230.83.203

#10: 157.230.83.203

#11: 77.120.163.23

#12: 77.120.163.23

#13: 77.120.163.23

#14: 77.120.163.23

#15: 77.120.163.23

#16: 77.120.163.23

#17: 77.120.163.23

#18: 77.120.163.23

#19: 77.120.163.23

#20: 77.120.163.23

#21: 77.120.163.87

#22: 77.120.163.87

#23: 77.120.163.87

#24: 77.120.163.87

#25: 77.120.163.87

#26: 77.120.163.87

#27: 77.120.163.87

#28: 77.120.163.87

#29: 77.120.163.87

#30: 77.120.163.87

#31: 178.128.210.1

#32: 178.128.210.1

#33: 178.128.210.1

#34: 178.128.210.1

#35: 178.128.210.1

#36: 178.128.210.1

#37: 178.128.210.1

#38: 178.128.210.1

#39: 178.128.210.1

#40: 178.128.210.1

#41: 77.120.163.23

#42: 77.120.163.23

#43: 77.120.163.23

#44: 77.120.163.23

#45: 77.120.163.23

#46: 77.120.163.23

#47: 77.120.163.23

#48: 77.120.163.23

#49: 77.120.163.23

#50: 7